In [2]:
from bertopic import BERTopic
#https://maartengr.github.io/BERTopic/getting_started/quickstart/quickstart.html 详细介绍文档
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
myfile = "/Volumes/时伟SSD512G/0717扶贫数据-已清洗完成-253267.xlsx"

In [3]:
df = pd.read_excel(myfile) #读取excel文件
df

,Unnamed: 0,date,time,username,place,clean_tweet,mentions,replies_count,retweets_count,likes_count,hashtags
0,0,2013-01-01,19:06:17,rotipeeta,NaN,awww poor help chinese restaurant block tmpt k...,[],0,0,0,[]
1,1,2013-01-01,23:13:13,waltydunlop,NaN,afraid thing think saw sky full chinese lanter...,[],0,0,0,[]
2,2,2013-01-01,05:13:11,mrjintheuk,NaN,battle damage clear tonight massive chinese me...,[],0,0,0,[]
3,3,2013-01-01,01:29:28,financenewscast,NaN,china economy end 2012 high 200m still live po...,[],0,0,0,[]
4,4,2013-01-01,05:21:28,warr1979,NaN,welcome another rescission government know mak...,[],0,0,0,[]
...,...,...,...,...,...,...,...,...,...,...,...
253262,253262,2021-12-31,13:58:29,ozzey_h,NaN,guarantee chinese government care people lift ...,[],0,0,0,[]
253263,253263,2021-12-31,17:25:30,redand51ue,NaN,shed crocodile tear chinese people force pover...,[],0,0,0,"['ccp', 'emperorxi', 'ccp', 'commonprosperity']"
253264,253264,2021-12-31,20:06:07,ray44758650,NaN,wrong deng wisely take ccp way chinese people ...,[],0,0,0,[]
253265,253265,2021-12-31,21:52:54,chineseemb_uga,NaN,excerpt president jinping 2022 new year addres...,[],1,2,9,[]


In [4]:
# Train BERTopic with a custom CountVectorizer
vectorizer_model = CountVectorizer(ngram_range=(1, 2), min_df=10)

In [ ]:
'''
单独把有进度条的那一部分，也就是词嵌入的转换，提取出来，这样之后就不用在花一个多小时重新算一遍了，节省时间
使用形式：topics, probs = topic_model.fit_transform(docs, embeddings)，embeddings直接载入参数部分

from sklearn.datasets import fetch_20newsgroups
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic

# Prepare embeddings
docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=False)

# Train our topic model using our pre-trained sentence-transformers embeddings
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(docs, embeddings)

'''

In [5]:
topic_model = BERTopic(vectorizer_model=vectorizer_model, language="english",
                       top_n_words=10, min_topic_size=50, 
                       calculate_probabilities=False, verbose=True, low_memory=True)

topics, probs = topic_model.fit_transform(df['clean_tweet'])

2022-07-17 11:11:58,654 - BERTopic - Transformed documents to Embeddings
OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2022-07-17 11:18:33,599 - BERTopic - Reduced dimensionality


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

2022-07-17 11:19:13,785 - BERTopic - Clustered reduced embeddings


In [39]:
#len(topics)
probs

array([0.32461287, 0.        , 0.76724741, ..., 1.        , 0.69705595,
       0.30111176])

In [6]:
freq = topic_model.get_topic_info()
freq

,Topic,Count,Name
0,-1,133291,-1_china_poor_people_chinese
1,0,5686,0_pakistan_pak_kashmir_modi
2,1,4342,1_india_china india_india china_indian
3,2,4175,2_capitalism_socialism_capitalist_communism
4,3,3171,3_africa_african_china africa_africa china
...,...,...,...
439,438,51,438_assistance china_work poverty_assistance_p...
440,439,51,439_oil_china oil_poor mark_discipline
441,440,51,440_china reach_milestone_end absolute_poverty...
442,441,50,441_aid_foreign aid_get aid_china aid


In [40]:
freq.to_excel("/Volumes/时伟SSD512G/topic_info442mac.xlsx")

In [41]:
df_topic = pd.DataFrame()
for i in range(len(freq)):
    df_topic.loc[:, i] = topic_model.get_topic(i)

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1667: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`



In [42]:
df_topic.to_excel("/Volumes/时伟SSD512G/topic_martix442mac.xlsx")

In [8]:
topic_model.get_topic(1)

[('india', 0.021686779879725535),
 ('china india', 0.010015479544669581),
 ('india china', 0.009115779236889412),
 ('indian', 0.00902145167637743),
 ('india poor', 0.005976396305196284),
 ('poverty india', 0.0057516937300276525),
 ('poor india', 0.0049984194378309855),
 ('growth', 0.004248531338377145),
 ('1962', 0.004095391823880741),
 ('population', 0.004041270476868729)]

In [25]:
representative_docs = topic_model.get_representative_docs(0)
print(representative_docs)

['imran khan announce subsidy scheme poor imf loan money give dole maybe get pay china military service afghanistan india china', 'china vow help pakistan eradicate poverty create job read arynews', 'please add list achievement nehru bharat divide religious ground nehru 1947 permanent seat offer china akshai chin area give china nehru excellent poor clothing ammunition provide military 1962']


In [26]:
frequency = topic_model.get_topic_freq(0)
print(frequency)

5686


In [48]:
#all_topics = topic_model.get_topics()
#print(all_topics)

In [9]:
#BERTopic.load("my_model")
topic_model.save("/Volumes/时伟SSD512G/bertopic_model_253267_0717")

/opt/anaconda3/lib/python3.9/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
#topic_model.get_topic(3)

In [17]:
topic_model.get_topic(256)

[('recovers', 0.28544684558924993),
 ('misuse poverty', 0.2768466208664615),
 ('misuse', 0.21183019714155774),
 ('relief fund', 0.18164495443262182),
 ('xinhua', 0.17234439106067426),
 ('xinhua china', 0.1716879792173356),
 ('poverty relief', 0.11546626706526578),
 ('fund', 0.103156619334901),
 ('relief', 0.09970013560678065),
 ('tech china', 0.0442148906536929)]

In [ ]:
#topic_model.get_topic(1)

In [28]:
#topic_model.visualize_topics()

In [23]:
#topic_model.visualize_barchart()

In [ ]:
#topic_model.visualize_distribution(probs[200], min_probability=0.0002)

In [18]:
#topic_model.visualize_hierarchy()

In [13]:
#topic_model.visualize_heatmap(n_clusters=10, width=1000, height=1000)

In [ ]:
#topic_model.visualize_term_rank()

In [35]:
#similar_topics, similarity = topic_model.find_topics("zhaolijian", top_n=5)
#print(similar_topics)
#print(similarity)

In [36]:
#topic_model.get_topic(31)

In [43]:
topics_over_time = topic_model.topics_over_time(df['clean_tweet'], topics, df['date'], 
                                                global_tuning=False, evolution_tuning=True, nr_bins=20)

20it [00:21,  1.09s/it]


In [45]:
#topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

In [47]:
df.insert(loc=5, column='topic', value=topics)

In [48]:
df

,Unnamed: 0,date,time,username,place,topic,clean_tweet,mentions,replies_count,retweets_count,likes_count,hashtags
0,0,2013-01-01,19:06:17,rotipeeta,NaN,9,awww poor help chinese restaurant block tmpt k...,[],0,0,0,[]
1,1,2013-01-01,23:13:13,waltydunlop,NaN,-1,afraid thing think saw sky full chinese lanter...,[],0,0,0,[]
2,2,2013-01-01,05:13:11,mrjintheuk,NaN,9,battle damage clear tonight massive chinese me...,[],0,0,0,[]
3,3,2013-01-01,01:29:28,financenewscast,NaN,-1,china economy end 2012 high 200m still live po...,[],0,0,0,[]
4,4,2013-01-01,05:21:28,warr1979,NaN,-1,welcome another rescission government know mak...,[],0,0,0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...
253262,253262,2021-12-31,13:58:29,ozzey_h,NaN,14,guarantee chinese government care people lift ...,[],0,0,0,[]
253263,253263,2021-12-31,17:25:30,redand51ue,NaN,4,shed crocodile tear chinese people force pover...,[],0,0,0,"['ccp', 'emperorxi', 'ccp', 'commonprosperity']"
253264,253264,2021-12-31,20:06:07,ray44758650,NaN,4,wrong deng wisely take ccp way chinese people ...,[],0,0,0,[]
253265,253265,2021-12-31,21:52:54,chineseemb_uga,NaN,17,excerpt president jinping 2022 new year addres...,[],1,2,9,[]


In [49]:
df.insert(loc=5, column='probs', value=probs)
df

,Unnamed: 0,date,time,username,place,probs,topic,clean_tweet,mentions,replies_count,retweets_count,likes_count,hashtags
0,0,2013-01-01,19:06:17,rotipeeta,NaN,0.324613,9,awww poor help chinese restaurant block tmpt k...,[],0,0,0,[]
1,1,2013-01-01,23:13:13,waltydunlop,NaN,0.000000,-1,afraid thing think saw sky full chinese lanter...,[],0,0,0,[]
2,2,2013-01-01,05:13:11,mrjintheuk,NaN,0.767247,9,battle damage clear tonight massive chinese me...,[],0,0,0,[]
3,3,2013-01-01,01:29:28,financenewscast,NaN,0.000000,-1,china economy end 2012 high 200m still live po...,[],0,0,0,[]
4,4,2013-01-01,05:21:28,warr1979,NaN,0.000000,-1,welcome another rescission government know mak...,[],0,0,0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
253262,253262,2021-12-31,13:58:29,ozzey_h,NaN,0.981416,14,guarantee chinese government care people lift ...,[],0,0,0,[]
253263,253263,2021-12-31,17:25:30,redand51ue,NaN,1.000000,4,shed crocodile tear chinese people force pover...,[],0,0,0,"['ccp', 'emperorxi', 'ccp', 'commonprosperity']"
253264,253264,2021-12-31,20:06:07,ray44758650,NaN,1.000000,4,wrong deng wisely take ccp way chinese people ...,[],0,0,0,[]
253265,253265,2021-12-31,21:52:54,chineseemb_uga,NaN,0.697056,17,excerpt president jinping 2022 new year addres...,[],1,2,9,[]


In [50]:
df.to_excel("/Volumes/时伟SSD512G/带topic和prob-0717扶贫数据-已清洗完成-253267.xlsx")